In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
#fname = 'files/Cal5DoF05222023-post FDPR V3.xlsx'
sheetname = 'Input for Tyler analysis'
fname = 'files/5DoF inputs - BA4C4.xlsx'

output_filename = 'Poses - BA4C8 FDPR update from BA4C4 - Evan.xlsx'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
         X    Y    Z  Rx  Ry
Name                       
NaN   NaN  NaN  NaN NaN NaN
NaN   NaN  NaN  NaN NaN NaN
NaN   NaN  NaN  NaN NaN NaN
NaN   NaN  NaN  NaN NaN NaN
NaN   NaN  NaN  NaN NaN NaN 

Provided 5DOF encoder values: 
                 X         Y         Z        Rx         Ry
Name.1                                                    
PR1       2.08241  -0.49623   0.75233 -0.076167  -0.006431
PR2    -108.30543 -47.63707  23.28538 -5.007865  10.641120
PR3     112.36741 -46.97690  23.34721 -4.976094 -10.638288
PR4     104.07263  58.08941  32.06823  5.193664 -10.358826
PR5    -100.30059  57.54782  31.83018  5.171731  10.361665


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: 
               X          Y          Z        Rx         Ry
PR1    2.104620  -0.533294   0.766874 -0.076922  -0.008507
PR2 -108.402137 -47.576225  23.393942 -5.007800  10.642134
PR3  112.414290 -47.270689  23.243794 -4.977458 -10.638489
PR4  104.264959  57.861268  31.875419  5.188853 -10.361210
PR5 -100.243632  57.672157  31.838508  5.172348  10.360847


### 0.3 Form a dataframe with the values supplied so far

In [5]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,Rz,uvec_X,uvec_Y,uvec_Z
PR1,2.104620,-0.533294,0.766874,-0.076922,-0.008507,crimson,NaN,-0.191849,250.541547,1267.600388
PR2,-108.402137,-47.576225,23.393942,-5.007800,10.642134,crimson,NaN,238.621698,352.402020,1220.022274
PR3,112.414290,-47.270689,23.243794,-4.977458,-10.638489,crimson,NaN,-238.540908,351.760084,1220.223310
PR4,104.264959,57.861268,31.875419,5.188853,-10.361210,crimson,NaN,-232.392471,130.977964,1264.286502
PR5,-100.243632,57.672157,31.838508,5.172348,10.360847,crimson,NaN,232.384416,131.342301,1264.250185
sMask,1.995000,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN,NaN
sMPA,2.040000,248.465000,1268.826000,-24.297100,0.007400,purple,0.0746,0.000129,0.411468,0.911424


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [6]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('files/FDPR shifts - sandbox.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                                ...                           
PR1                    -87                -89  261.2  ...     -5.2      -51      -52
PR3                   -250               -258  260.0  ...     -4.0     -149      -40
PR4                    -13                -14  264.9  ...     -8.9      -71      -89
PR5                    -19                -21  268.3  ...    -12.3      -81     -123
PR2                   -137               -141  252.1  ...      3.9     -122       39

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [7]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

## Scratch

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 5DOF optimization)

In [8]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals (in sMPA frame) baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.050919  0.064709  0.065881
PR2  0.148090  0.003048  0.149100
PR3  0.101318  0.106862  0.227088
PR4  0.068366  0.082616 -0.022969
PR5  0.084617  0.114332 -0.030116 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [-0.0523  1.1834 -0.1381  0.0509  0.0064] 

Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.042    0.008   -0.018
PR2    0.058   -0.088    0.000
PR3    0.011    0.001    0.027
PR4   -0.022    0.021   -0.027
PR5   -0.006    0.058    0.018 

endpt_X    0.033868
endpt_Y    0.048142
endpt_Z    0.020501
dtype: float64


### 1.2 (alternate) Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [9]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.050919  0.064709  0.065881
PR2  0.148090  0.003048  0.149100
PR3  0.101318  0.106862  0.227088
PR4  0.068366  0.082616 -0.022969
PR5  0.084617  0.114332 -0.030116 

Best-fit 6DoF deltas to apply to real-space poses: 
 [-0.0522  1.1835 -0.1381  0.0509  0.0055 -0.005 ] 

Residuals after incorporating the best-fit 6DoF shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.042    0.008   -0.018
PR2    0.048   -0.068   -0.004
PR3    0.002   -0.019    0.031
PR4   -0.012    0.001   -0.023
PR5    0.004    0.078    0.014 

endpt_X    0.029219
endpt_Y    0.047148
endpt_Z    0.020124
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [10]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
               X          Y          Z        Rx         Ry
PR1    2.028079   0.683438   0.617852 -0.025234    0.00001
PR2 -108.359271 -46.455617  23.148959 -4.955955  10.647996
PR3  112.312465   -45.7896  23.208192 -4.925982 -10.631409
PR4  104.018414  59.264746  31.937823  5.243735 -10.352819
PR5 -100.353748  58.717858  31.702055   5.22361  10.367667 

Baseline pose encoders (displayed again here just for reference): 
                 X         Y         Z        Rx         Ry
Name.1                                                    
PR1       2.08241  -0.49623   0.75233 -0.076167  -0.006431
PR2    -108.30543 -47.63707  23.28538 -5.007865  10.641120
PR3     112.36741 -46.97690  23.34721 -4.976094 -10.638288
PR4     104.07263  58.08941  32.06823  5.193664 -10.358826
PR5    -100.30059  57.54782  31.83018  5.171731  10.361665


# 2 Write the new poses to an Excel file in the current directory

In [11]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='FDPR',baseline_filepath=fname,
                             columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,rigid_body_correction=best_fit_deltas,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)

**Writing to Excel complete.**
**Filename:  Poses - BA4C8 FDPR update from BA4C4 - Evan.xlsx
